In [1]:
import ibis
from pathlib import Path
import ibis_widget
import election_data as ed
from ibis.expr import types as ir

ibis_widget.install()
ibis.options.interactive = True

from ibis import _

In [2]:
# Use this as reference for what MEDSL did previously, since their replication
# scripts are insufficient.
gold = ibis.read_parquet("../data/cleaned.parquet")
gold_va = gold.filter(_.state_po == "VA")
gold_va

┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ year  ┃ date       ┃ state_po ┃ county_name ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ district ┃ office                                       ┃ magnitude ┃ special ┃ stage  ┃ precinct                              ┃ writein ┃ candidate ┃ party_detailed ┃ party_simplified ┃ mode   ┃ votes  ┃ readme_check ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━┩
│ int16 │ date       │ string   │ string      │ string      │ string            │ string            │ string   │ string                                       │ int64     │ boolean │ string │ string                                │ boolean │ string    │ string         │ string           │ string │ string │ boolean      │
├───────┼────────────┼──────────┼─────────────┼─────────────┼───────────────────┼───────────────────┼──────────┼──────────────────────────────────────────────┼───────────┼─────────┼────────┼───────────────────────────────────────┼─────────┼───────────┼────────────────┼──────────────────┼────────┼────────┼──────────────┤
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ # AB - CENTRAL ABSENTEE PRECINCT (02) │ False   │ NO        │ NULL           │ NULL             │ TOTAL  │ 234    │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ # AB - CENTRAL ABSENTEE PRECINCT (02) │ False   │ YES       │ NULL           │ NULL             │ TOTAL  │ 887    │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ ## PROVISIONAL (02)                   │ False   │ NO        │ NULL           │ NULL             │ TOTAL  │ 0      │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ ## PROVISIONAL (02)                   │ False   │ YES       │ NULL           │ NULL             │ TOTAL  │ 2      │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ 101 - CHINCOTEAGUE (02)               │ False   │ NO        │ NULL           │ NULL             │ TOTAL  │ 285    │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ 101 - CHINCOTEAGUE (02)               │ False   │ YES       │ NULL           │ NULL             │ TOTAL  │ 1098   │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │         1 │ False   │ GEN    │ 201 - ATLANTIC (02)                   │ False   │ NO        │ NULL           │ NULL             │ TOTAL  │ 202    │ False        │
│  2018 │ 2018-11-06 │ VA       │ ACCOMACK    │ 51001       │ ACCOMACK          │ 51001             │ NULL     │ PROPOSED CONSTITUTIONAL

In [3]:
# see https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/
# There are other races available, such as primaries.
urls = {
    2017: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2017/2017%20November%20General.csv",
    2019: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2019/2019%20November%20General.csv",
    2021: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2021/2021%20November%20General%20.csv",
    # 2023 isn't available yet :( https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2023/
}
paths = {year: Path(f"../data/va/{year}.csv") for year in urls}
for year, url in urls.items():
    path = paths[year]
    path.parent.mkdir(parents=True, exist_ok=True)
    if not path.exists():
        print(f"Downloading {year} results to {path}")
        !curl -o {path} {url}

In [4]:
raws = [ibis.read_csv(paths[year]).mutate(year=year) for year in urls]
raw = ibis.union(*raws)
raw.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ __row_id ┃ CandidateUid                           ┃ FirstName ┃ MiddleName ┃ LastName       ┃ Suffix ┃ TOTAL_VOTES ┃ Party       ┃ WriteInVote ┃ LocalityUid                            ┃ LocalityCode ┃ LocalityName    ┃ PrecinctUid ┃ PrecinctName   ┃ DistrictUid                            ┃ DistrictType       ┃ DistrictName ┃ OfficeUid                              ┃ OfficeTitle                     ┃ ElectionUid                            ┃ ElectionType ┃ ElectionDate        ┃ Ele

In [5]:
t = raw
assert (t.ElectionType == "General").all().execute()
t = t.drop("ElectionType")
t = t.drop("ElectionName")

t = t.drop(
    "CandidateUid",
    "LocalityUid",
    "LocalityCode",
    "PrecinctUid",
    "DistrictUid",
    "OfficeUid",
    "ElectionUid",
)
# In the larger dataset, the vote counts can be the string "*",
# so we need the votes column to be a string to match that.
t = t.mutate(votes=_.TOTAL_VOTES.cast(str)).drop("TOTAL_VOTES")
t = t.mutate(date=_.ElectionDate.date()).drop("ElectionDate")
t = t.mutate(
    readme_check=ibis.literal(False),
    special=ibis.literal(False),
    stage=ibis.literal(ed.Stage.GEN),
)

assert (t.date.nunique()).execute() == len(urls)
t = t.cache()
t.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ __row_id ┃ FirstName ┃ MiddleName ┃ LastName       ┃ Suffix ┃ Party       ┃ WriteInVote ┃ LocalityName    ┃ PrecinctName   ┃ DistrictType       ┃ DistrictName ┃ OfficeTitle                     ┃ year  ┃ votes  ┃ date       ┃ readme_check ┃ special ┃ stage  ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ int64    │ string    │ string     │ string         │ string │ string      │ int64       │ string          │ string         │ string             │ string       │ string                          │ int16 │ string │ dat

## fix county and county_fips

In [6]:
def compare_values(
    a: ir.StringValue, b: ir.StringValue, *, norm: bool = True
) -> ir.BooleanValue:
    from IPython.display import display

    if norm:
        a = a.upper().strip()
        b = b.upper().strip()
    a = a.name("val")
    b = b.name("val")
    vc_a = a.value_counts(name="n_a")
    vc_b = b.value_counts(name="n_b")
    vc = vc_a.outer_join(vc_b, "val", lname="{name}_a", rname="{name}_b")
    w1 = vc.order_by(_.n_a.desc())
    w2 = vc.order_by(_.n_b.desc())
    display(w1)
    display(w2)


compare_values(t.LocalityName, gold_va.county_name)

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ val_a                 ┃ n_a   ┃ val_b               ┃ n_b   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string                │ int64 │ string              │ int64 │
├───────────────────────┼───────┼─────────────────────┼───────┤
│ FAIRFAX COUNTY        │ 15552 │ NULL                │  NULL │
│ VIRGINIA BEACH CITY   │  8349 │ VIRGINIA BEACH CITY │  3282 │
│ LOUDOUN COUNTY        │  6883 │ NULL                │  NULL │
│ PRINCE WILLIAM COUNTY │  6233 │ NULL                │  NULL │
│ CHESTERFIELD COUNTY   │  5231 │ NULL                │  NULL │
│ HENRICO COUNTY        │  5074 │ NULL                │  NULL │
│ RICHMOND CITY         │  4230 │ RICHMOND CITY       │  1765 │
│ CHESAPEAKE CITY       │  4021 │ CHESAPEAKE CITY     │  3864 │
│ ARLINGTON COUNTY      │  3536 │ NULL                │  NULL │
│ NORFOLK CITY          │  3470 │ NORFOLK CITY        │  1610 │
│ …                     │     … │ …                   │     … │
└───────────────────────┴───────┴─────────────────────┴───────┘

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ val_a               ┃ n_a   ┃ val_b               ┃ n_b   ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string              │ int64 │ string              │ int64 │
├─────────────────────┼───────┼─────────────────────┼───────┤
│ NULL                │  NULL │ FAIRFAX             │  6353 │
│ CHESAPEAKE CITY     │  4021 │ CHESAPEAKE CITY     │  3864 │
│ VIRGINIA BEACH CITY │  8349 │ VIRGINIA BEACH CITY │  3282 │
│ NULL                │  NULL │ LOUDOUN             │  2841 │
│ NULL                │  NULL │ PRINCE WILLIAM      │  2702 │
│ NULL                │  NULL │ HENRICO             │  2426 │
│ NULL                │  NULL │ CHESTERFIELD        │  2169 │
│ NULL                │  NULL │ ARLINGTON           │  1811 │
│ NEWPORT NEWS CITY   │  2636 │ NEWPORT NEWS CITY   │  1808 │
│ RICHMOND CITY       │  4230 │ RICHMOND CITY       │  1765 │
│ …                   │     … │ …                   │     … │
└─────────────────────┴───────┴─────────────────────┴───────┘

In [7]:
def make_county_fips_lookup() -> ibis.Table:
    t = ibis.read_parquet("../data/cleaned.parquet")
    return t.select(
        "state_po",
        "county_name",
        "county_fips",
    ).distinct()


def add_county_fips(t: ibis.Table) -> ibis.Table:
    lookup = make_county_fips_lookup()
    return t.left_join(lookup, ["state_po", "county_name"]).drop(
        "state_po_right", "county_name_right"
    )


t = t.mutate(
    state_po=ibis.literal("VA"),
    county_name=_.LocalityName.upper().replace(" & ", " AND ").replace(" COUNTY", ""),
).drop("LocalityName")
t = add_county_fips(t)
# check we found a fips code for every county.
assert (t.county_fips.notnull()).all().execute()

## Fix jurisdiction and jurisdiction_fips

Looks like in the upstream data, jurisdiction is always the same as
the county. So just use that.

Be warned, it appears that the upstream FIPS codes for jurisdictions are somewhat BS.
For example, the "DISTRICT 12" jurisdiction in alaska has a FIPS code of "02012"
which isn't a [real fips code](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt)
FIPS codes really only go down to county level. Probably this DISTRICT 12
is actually referring to state HD 12, even though it is filed under the
US HOUSE office:

In [8]:
assert (gold_va.county_name == gold_va.jurisdiction_name).all().execute()
assert (gold_va.county_fips == gold_va.jurisdiction_fips).all().execute()
t = t.mutate(
    jurisdiction_name=_.county_name,
    jurisdiction_fips=_.county_fips,
)


## Add candidate and writein

In [9]:
t = t.mutate(
    candidate=(
        _.FirstName.fill_null("")
        + " "
        + _.MiddleName.fill_null("")
        + " "
        + _.LastName.fill_null("")
        + " "
        + _.Suffix.fill_null("")
    )
    .re_replace(r"\s+", " ")
    .strip()
    .upper(),
).drop("FirstName", "MiddleName", "LastName", "Suffix")

assert (t.filter(_.WriteInVote != 0).candidate == "WRITE IN VOTES").all().execute()
assert (t.filter(_.WriteInVote == 0).candidate != "WRITE IN VOTES").all().execute()
assert (t.filter(_.candidate == "WRITE IN VOTES").WriteInVote == 1).all().execute()
assert (t.filter(_.candidate != "WRITE IN VOTES").WriteInVote == 0).all().execute()

t = t.mutate(
    writein=_.WriteInVote == 1,
    candidate=ibis.case().when(_.WriteInVote == 1, "WRITEIN").else_(_.candidate).end(),
).drop("WriteInVote")
t.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ __row_id ┃ Party       ┃ PrecinctName            ┃ DistrictType       ┃ DistrictName ┃ OfficeTitle                                                     ┃ year  ┃ votes  ┃ date       ┃ readme_check ┃ special ┃ stage  ┃ state_po ┃ county_name       ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ candidate                   ┃ writein ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━

## Fix Party

In [10]:
# when party is write-in, that isn't a writein *candidate*:
# Their name actually appeared on the ballot, but they chose to
# use the write-in party.
assert (~t.filter(_.Party == "Write-In").writein).all().execute()

In [11]:
party_mapping = {
    "Democratic": ("DEMOCRAT", "DEMOCRAT"),
    "Republican": ("REPUBLICAN", "REPUBLICAN"),
    "Libertarian": ("LIBERTARIAN", "LIBERTARIAN"),
    "Liberation": ("OTHER", "LIBERATION"),
    "Green": ("GREEN", "GREEN"),
    "Independent": ("INDEPENDENT", "INDEPENDENT"),
    "Write-In": ("OTHER", None),
}
# Verify the values in the column are always one of the above
assert t.Party.isin(list(party_mapping)).all().execute()
simplified = ed.util.cases(*[(t.Party == k, v[0]) for k, v in party_mapping.items()])
detailed = ed.util.cases(*[(t.Party == k, v[1]) for k, v in party_mapping.items()])
t = t.mutate(party_simplified=simplified, party_detailed=detailed).drop("Party")
ed.tests.test_party(t)

## Precinct check

It looks like there isn't anything crazy going on here. Just normalize the strings and call it good.

In [12]:
vc_gold = gold_va.precinct.upper().name("precinct").value_counts(name="n_gold")
vc_t = t.PrecinctName.upper().name("precinct").value_counts(name="n_t")
vc = vc_gold.outer_join(vc_t, "precinct")
vc.order_by(_.n_gold.desc()).widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ __row_id ┃ precinct                         ┃ n_gold ┃ precinct_right                   ┃ n_t   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ int64    │ string                           │ int64  │ string                           │ int64 │
├──────────┼──────────────────────────────────┼────────┼──────────────────────────────────┼───────┤
│        0 │ ## PROVISIONAL                   │   4205 │ ## PROVISIONAL                   │ 12245 │
│        1 │ # AB - CENTRAL ABSENTEE PRECINCT │   2446 │ # AB - CENTRAL ABSENTEE PRECINCT │  8265 │
│        2 │ ##EV - CENTRAL ABSENTEE PRECINCT │   2384 │ ##EV - CENTRAL ABSENTEE PRECINCT │  3742 │
│        3 │ ##AB - CENTRAL ABSENTEE PRECINCT │   2376 │ ##AB - CENTRAL ABSENTEE PRECINCT │  3740 │
│        4 │ ##PE - CENTRAL ABSENTEE PRECINCT │   2241 │ ##PE - CENTRAL ABSENTEE PRECINCT │  3738 │


In [13]:
vc.order_by(_.n_t.desc()).widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ __row_id ┃ precinct                         ┃ n_gold ┃ precinct_right                   ┃ n_t   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ int64    │ string                           │ int64  │ string                           │ int64 │
├──────────┼──────────────────────────────────┼────────┼──────────────────────────────────┼───────┤
│        0 │ ## PROVISIONAL                   │   4205 │ ## PROVISIONAL                   │ 12245 │
│        1 │ # AB - CENTRAL ABSENTEE PRECINCT │   2446 │ # AB - CENTRAL ABSENTEE PRECINCT │  8265 │
│        2 │ ##EV - CENTRAL ABSENTEE PRECINCT │   2384 │ ##EV - CENTRAL ABSENTEE PRECINCT │  3742 │
│        3 │ ##AB - CENTRAL ABSENTEE PRECINCT │   2376 │ ##AB - CENTRAL ABSENTEE PRECINCT │  3740 │
│        4 │ ##PE - CENTRAL ABSENTEE PRECINCT │   2241 │ ##PE - CENTRAL ABSENTEE PRECINCT │  3738 │


In [14]:
t = t.mutate(precinct=_.PrecinctName.upper().strip().replace(" & ", " AND ")).drop(
    "PrecinctName"
)

In [15]:
ed.tests.test_schema(t)

SchemaMismatchError: Extra columns: {'OfficeTitle', 'DistrictType', 'DistrictName'}
Missing columns: {'district', 'mode', 'office', 'magnitude'}
Conflicting columns: set()
Columns out of order: set()

## Fix office

In [20]:
compare_values(gold_va.office, t.OfficeTitle)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ val_a                                        ┃ n_a   ┃ val_b                         ┃ n_b   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string                                       │ int64 │ string                        │ int64 │
├──────────────────────────────────────────────┼───────┼───────────────────────────────┼───────┤
│ US HOUSE                                     │ 26558 │ NULL                          │  NULL │
│ US SENATE                                    │ 19122 │ NULL                          │  NULL │
│ US PRESIDENT                                 │ 11100 │ NULL                          │  NULL │
│ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 2 │  5510 │ NULL                          │  NULL │
│ PROPOSED CONSTITUTIONAL AMENDMENT QUESTION 1 │  5510 │ NULL                          │  NULL │
│ MEMBER CITY COUNCIL                          │  3491 │ MEMBER CITY COUNCIL           │   986 │
│ MEMBER SCHOOL BOARD                          │  3318 │ MEMBER SCHOOL BOARD           │ 10764 │
│ MEMBER TOWN COUNCIL                          │  1973 │ MEMBER TOWN COUNCIL           │   966 │
│ MAYOR                                        │   928 │ MAYOR                         │   466 │
│ MEMBER SCHOOL BOARD - SPECIAL                │   392 │ MEMBER SCHOOL BOARD - SPECIAL │   149 │
│ …                                            │     … │ …                             │     … │
└──────────────────────────────────────────────┴───────┴───────────────────────────────┴───────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ val_a                       ┃ n_a   ┃ val_b                       ┃ n_b   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string                      │ int64 │ string                      │ int64 │
├─────────────────────────────┼───────┼─────────────────────────────┼───────┤
│ NULL                        │  NULL │ MEMBER HOUSE OF DELEGATES   │ 28390 │
│ NULL                        │  NULL │ GOVERNOR                    │ 23332 │
│ NULL                        │  NULL │ LIEUTENANT GOVERNOR         │ 17499 │
│ NULL                        │  NULL │ ATTORNEY GENERAL            │ 17499 │
│ MEMBER SCHOOL BOARD         │  3318 │ MEMBER SCHOOL BOARD         │ 10764 │
│ SHERIFF                     │    55 │ SHERIFF                     │ 10085 │
│ MEMBER BOARD OF SUPERVISORS │   279 │ MEMBER BOARD OF SUPERVISORS │  9933 │
│ COMMONWEALTH'S ATTORNEY     │   246 │ COMMONWEALTH'S ATTORNEY     │  8332 │
│ NULL                        │  NULL │ MEMBER SENATE OF VIRGINIA   │  8314 │
│ NULL                        │  NULL │ TREASURER                   │  7994 │
│ …                           │     … │ …                           │     … │
└─────────────────────────────┴───────┴─────────────────────────────┴───────┘

In [21]:
t = t.mutate(office=_.OfficeTitle.upper().strip()).drop("OfficeTitle")
ed.tests.test_schema(t)

SchemaMismatchError: Extra columns: {'DistrictType', 'DistrictName'}
Missing columns: {'district', 'mode', 'magnitude'}
Conflicting columns: set()
Columns out of order: set()

In [22]:
gold_va.filter(
    _.mode != "ELECTION DAY",
    # _.mode == "ELECTION DAY",
    _.mode != "TOTAL",
).select("mode", "precinct").value_counts().execute()

,mode,precinct,mode_precinct_count
0,ABSENTEE,# AB - CENTRAL ABSENTEE PRECINCT,2446
1,IN-PERSON,104 - WILSON,6
2,IN-PERSON,201 - CRIMORA,6
3,IN-PERSON,203 - WEYERS CAVE,6
4,IN-PERSON,302 - NORTH RIVER,6
...,...,...,...
2479,IN-PERSON,602 - MENDOTA,3
2480,IN-PERSON,001 - WARD A,3
2481,IN-PERSON,002 - WARD B,3
2482,IN-PERSON,502 - SHEFFEY,3


In [29]:
gold_va.mode.value_counts()

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ mode         ┃ mode_count ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string       │ int64      │
├──────────────┼────────────┤
│ PROVISIONAL  │       4205 │
│ ABSENTEE     │       9447 │
│ TOTAL        │      11020 │
│ ELECTION DAY │      42344 │
│ IN-PERSON    │      17270 │
└──────────────┴────────────┘

In [11]:
t = t.mutate(
    district=(t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE").ifelse(
        "AT-LARGE", t.district
    ),
    office=(t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE").ifelse(
        "MEMBER BOARD OF SUPERVISORS", t.office
    ),
)

AttributeError: 'Table' object has no attribute 'office'

In [49]:
ed.tests.test_schema(t)

SchemaMismatchError: Extra columns: {'DistrictName', 'DistrictType', 'OfficeTitle', 'PrecinctName'}
Missing columns: {'district', 'jurisdiction_name', 'precinct', 'stage', 'mode', 'magnitude', 'office', 'jurisdiction_fips'}
Conflicting columns: {('votes', Int64(nullable=True), String(nullable=True))}
Columns out of order: set()

In [41]:
gold_va.select("magnitude", "office").value_counts().order_by(
    _.magnitude.desc()
).widget()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ __row_id ┃ magnitude ┃ office                           ┃ magnitude_office_count ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64    │ int64     │ string                           │ int64                  │
├──────────┼───────────┼──────────────────────────────────┼────────────────────────┤
│        0 │         7 │ MEMBER TOWN COUNCIL              │                    137 │
│        1 │         7 │ MEMBER TOWN COUNCIL - JONESVILLE │                     65 │
│        2 │         6 │ MEMBER TOWN COUNCIL              │                    527 │
│        3 │         6 │ MEMBER TOWN COUNCIL - SAXIS      │                     33 │
│        4 │         6 │ MEMBER CITY COUNCIL              │                     99 │
│        5 │         6 │ MEMBER TOWN COUNCIL - ACCOMAC    │                     32 │
│        6 │         6 │ MEMBER TOWN COUNCIL - EASTVILLE  │      

In [13]:
# ported from https://github.com/MEDSL/replication-scripts/blob/634832bc7c2df2c0a2238bc16781d07d581d3a3a/va2018.py


# Fill NA values with an empty string
t = t.fill_null("")

# Replace specific county name
t = t.mutate(
    county_name=ibis.case()
    .when(t.county_name == "KING & QUEEN", "KING AND QUEEN")
    .else_(t.county_name)
    .end()
)

# Load and merge FIPS codes
fips = con.table("../../../help-files/county-fips-codes.csv")
fips = fips.mutate(state=fips.state.upper())
t = t.join(fips, ["state", "county_name"], how="left")
t = t.mutate(county_fips=t.county_fips.cast("string").zfill(5))

# Load jurisdiction FIPS and merge based on conditions
juris_fips = con.table("../../../help-files/jurisdiction-fips-codes.csv")
juris_fips = juris_fips.mutate(state=juris_fips.state.upper())

states_w_juris = (
    juris_fips.filter(juris_fips.jurisdiction_fips.length() > 5)
    .state.distinct()
    .to_list()
)
if t.state.distinct().execute()[0] not in states_w_juris:
    t = t.mutate(jurisdiction_fips=t.county_fips, jurisdiction_name=t.county_name)
else:
    juris_fips = juris_fips.mutate(
        county_fips=juris_fips.jurisdiction_fips.str.zfill(10).substr(0, 5)
    )
    t = t.join(juris_fips, ["state", "county_fips", "jurisdiction_name"], how="left")
    t = t.mutate(
        jurisdiction_fips=ibis.ifelse(
            t.jurisdiction_fips.isnull(),
            "Missing Jurisdiction FIPS",
            t.jurisdiction_fips,
        )
    )

# Pad and clean district codes
t = t.mutate(
    district=t.district.zfill(3).replace(
        {"00A": "A", "00B": "B", "00C": "C", "000": ""}
    )
)

# Set date and readme_check values
t = t.mutate(date="2018-11-06", readme_check="FALSE")

# Merge magnitudes data
mags = con.table("magnitudes.csv").fillna("")
t = t.join(mags, ["office", "jurisdiction_name", "a"], how="left")
t = t.mutate(magnitude=t.magnitude.fillna(1).cast("int64"))

# Write-in candidate replacements
t = t.mutate(
    candidate=t.candidate.replace("[WRITE-IN]", "WRITEIN", regex=False).replace(
        r"\.", "", regex=True
    )
)

# Apply specific conditions for certain office values
t = t.mutate(
    district=ibis.case()
    .when(t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE", "AT-LARGE")
    .else_(t.district)
    .end(),
    office=ibis.case()
    .when(
        t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE",
        "MEMBER BOARD OF SUPERVISORS",
    )
    .else_(t.office)
    .end(),
)

# Load amendments data and concatenate
ammendments = con.table("va-18-ammendments.csv", schema=official_dtypes)
t = t.union(ammendments)

# Replace boolean values with string equivalents and trim extra spaces
t = t.replace({True: "TRUE", False: "FALSE"})
t = t.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Export to CSV
t.to_csv(
    "2018-va-precinct-general-updated.csv", quoting=csv.QUOTE_NONNUMERIC, index=False
)


IbisTypeError: Cannot fill_null on column 'year' of type int16 with a value of type string - pass in an explicit mapping of fill values to `fill_null` instead.